In [1]:
import fastai
from fastai.vision import *
from fastai.callbacks import *
from fastai.torch_core import *
from fastai.callback import *
from fastai.basic_train import *
import torchvision.models as tmodels

import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from sklearn.metrics import accuracy_score
from scipy.misc import imsave

from sklearn.metrics import accuracy_score, auc, roc_auc_score, roc_curve
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score
from scipy.misc import imsave

import torchvision
from torch.nn import Sigmoid
from torch import Tensor, LongTensor
from torch.autograd import Variable
import torch.nn as nn
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.nn import Sequential
from torchvision.models.resnet import resnet50





import os
import sys

sys.path.insert(0, '/home/santhosr/Documents/Birad/FastAI/RaceClassification/' )

from utils import *

from birad import *
from birad import setTruthFile, getRaceLabel


In [2]:
inputFolder1 = '/home/santhosr/Documents/Birad/ProcessedData/FullRes'
truthFile1 = '/home/santhosr/Documents/Birad/birad_targetFile.csv'

inputFolder2 = '/home/santhosr/Documents/Birad/ProcessedData/PennExtra_3500/'
truthFile2 = '/home/santhosr/Documents/Birad/RaceDL_ExtraCaucasian.csv'

df1 = pd.read_csv('/home/santhosr/Documents/Birad/birad_targetFile.csv')
df1.drop(['PresIntentType','DBT'],inplace = True,axis=1)


df2 = pd.read_csv('/home/santhosr/Documents/Birad/RaceDL_ExtraCaucasian.csv')
df2.Medview_Race = 'White'
truth = pd.concat([df1,df2],sort=True)

setTruthFile(truth)


dfFile = "/home/santhosr/Documents/Birad/FastAI/RaceClassification/withLargerDataset/DataFile10.csv"

modelName = 'model_resnet50_id10_acc833_loss386'


#Feature Directories
trainFolder = '/home/santhosr/Documents/Birad/FastAI/RaceClassification/withLargerDataset/Features/model9_2/train'

validFolder = '/home/santhosr/Documents/Birad/FastAI/RaceClassification/withLargerDataset/Features/model9_2/valid'

df = pd.read_csv(dfFile)

In [3]:
d = pd.concat([df[df.train==True].iloc[0:100], df[df.train==False].iloc[0:100], ])

#Creates the FastAI Dataset
data = ImageItemList.from_df(df=d,path='/home/santhosr/Documents/Birad/ProcessedData/', cols='filename').split_from_df(col='train').label_from_func(getRaceLabel).transform(get_transforms(),size=256).databunch(bs=50).normalize()
print("Dataset created")

Dataset created


### FastAI Model

In [4]:
#Creates the model architecture 
learn = create_cnn(data, tmodels.resnet50, metrics=accuracy,pretrained=True)

learn.load('/home/santhosr/Documents/Birad/ProcessedData/models/'+modelName)



Learner(data=ImageDataBunch;

Train: LabelList
y: CategoryList (100 items)
[Category 0, Category 1, Category 0, Category 1, Category 0]...
Path: /home/santhosr/Documents/Birad/ProcessedData
x: ImageItemList (100 items)
[Image (3, 3328, 2560), Image (3, 4096, 3328), Image (3, 4096, 3328), Image (3, 3328, 2560), Image (3, 3328, 2560)]...
Path: /home/santhosr/Documents/Birad/ProcessedData;

Valid: LabelList
y: CategoryList (100 items)
[Category 1, Category 1, Category 1, Category 0, Category 0]...
Path: /home/santhosr/Documents/Birad/ProcessedData
x: ImageItemList (100 items)
[Image (3, 4096, 3328), Image (3, 4096, 3328), Image (3, 3328, 2560), Image (3, 4096, 3328), Image (3, 3328, 2560)]...
Path: /home/santhosr/Documents/Birad/ProcessedData;

Test: None, model=Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
 

### PyTorch Model

In [13]:
class fastAIModel(nn.Module):
    def __init__(self):
        super(fastAIModel, self).__init__()
        
    
        self.body =  Sequential(*list(resnet50().children())[:-2])
    
        
        self.head =     Sequential(
                            AdaptiveConcatPool2d(), Flatten(),
                            nn.BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), 
                            nn.Dropout(p=0.25), 
                            nn.Linear(in_features=4096, out_features=512, bias=True), 
                            nn.ReLU(inplace = True),
                            nn.BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), 
                            nn.Dropout(p=0.5), 
                            nn.Linear(in_features=512, out_features=2, bias=True)
                        )
        
        
        self.total = nn.Sequential(self.body, self.head)
    
        
    def forward(self, x):
        
        
        x = self.total(x)
        

        return x
    

model = fastAIModel()

In [14]:
model.total.load_state_dict(torch.load('/home/santhosr/Documents/Birad/ProcessedData/models/model_resnet50_id10_acc833_loss386.pth')['model'])



### Testing on common matrices

In [5]:
t = torch.ones((1,3,224,224))


learn.model.eval()
learn.model(t.cuda())

tensor([[-0.0685,  0.5951]], device='cuda:0', grad_fn=<AddmmBackward>)

In [ ]:
model.eval()
model(t)

In [24]:
rand = torch.rand((1,3,224,224))

learn.model.eval()
learn.model(rand.cuda())

tensor([[ 1.8616, -1.6184]], device='cuda:0', grad_fn=<AddmmBackward>)

In [25]:
model.eval()
model(rand)

tensor([[ 1.8616, -1.6184]], grad_fn=<AddmmBackward>)

### Testing on images

In [6]:
img = data.valid_ds.x[0]

In [32]:
print(img.data.dtype)
img.data

torch.float32


tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]]])

In [8]:
learn.predict(img)

(Category 1, tensor(1), tensor([0.0057, 0.9943]))

In [9]:
processedImg = learn.data.one_item(data.valid_dl.x[0])[0]

processedImg.shape

torch.Size([1, 3, 256, 256])

In [10]:
learn.model(processedImg)

tensor([[-2.8241,  2.3344]], device='cuda:0', grad_fn=<AddmmBackward>)

In [11]:
nn.Softmax()(learn.model(processedImg))

/home/santhosr/.conda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


tensor([[0.0057, 0.9943]], device='cuda:0', grad_fn=<SoftmaxBackward>)

##### 1

In [24]:
def pil2tensor(image,dtype):
    "Convert PIL style `image` array to torch style image tensor."
    a = np.asarray(image)
    if a.ndim==2 : a = np.expand_dims(a,2)
    a = np.transpose(a, (1, 0, 2))
    a = np.transpose(a, (2, 1, 0))
    return torch.from_numpy(a.astype(dtype, copy=False) )

filename = '/home/santhosr/Documents/Birad/ProcessedData/./FullRes/2/75048084_R_CC_1.jpg'

x = PIL.Image.open(filename).convert('RGB')

x = pil2tensor(x,np.float32)

x.div_(255)   

out = fastai.vision.Image(x)

In [25]:
tp = data.one_item(out)

In [26]:
nn.Softmax()(learn.model(tp[0]))

/home/santhosr/.conda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


tensor([[0.0057, 0.9943]], device='cuda:0', grad_fn=<SoftmaxBackward>)

##### 2

In [28]:
def pil2tensor(image,dtype):
    "Convert PIL style `image` array to torch style image tensor."
    a = np.asarray(image)
    if a.ndim==2 : a = np.expand_dims(a,2)
    a = np.transpose(a, (1, 0, 2))
    a = np.transpose(a, (2, 1, 0))
    return torch.from_numpy(a.astype(dtype, copy=False) )

filename = '/home/santhosr/Documents/Birad/ProcessedData/./FullRes/2/75048084_R_CC_1.jpg'

x = PIL.Image.open(filename).convert('RGB')

x = pil2tensor(x,np.float32)

x.div_(255)   

out = fastai.vision.Image(x)

ds = data.single_ds

with ds.set_item(out):
    batch =  data.one_batch(ds_type=DatasetType.Single, detach=False, denorm=False)

In [29]:
nn.Softmax()(learn.model(batch[0]))

/home/santhosr/.conda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


tensor([[0.0057, 0.9943]], device='cuda:0', grad_fn=<SoftmaxBackward>)

##### 3

#### Tests

In [9]:
import pdb
import inspect

import PIL

In [12]:
pdb.run('learn.predict(data.valid_dl.x[1])')

> <string>(1)<module>()->None
(Pdb) s
--Call--
> /home/santhosr/.local/lib/python3.6/site-packages/fastai/basic_data.py(35)__getattr__()
-> def __getattr__(self,k:str)->Any: return getattr(self.dl, k)
(Pdb) n
> /home/santhosr/.local/lib/python3.6/site-packages/fastai/basic_data.py(35)__getattr__()
-> def __getattr__(self,k:str)->Any: return getattr(self.dl, k)
(Pdb) n
--Return--
> /home/santhosr/.local/lib/python3.6/site-packages/fastai/basic_data.py(35)__getattr__()->ImageItemList.../ProcessedData
-> def __getattr__(self,k:str)->Any: return getattr(self.dl, k)
(Pdb) n
--Call--
> /home/santhosr/.local/lib/python3.6/site-packages/fastai/data_block.py(92)__getitem__()
-> def __getitem__(self,idxs:int)->Any:
(Pdb) n
> /home/santhosr/.local/lib/python3.6/site-packages/fastai/data_block.py(93)__getitem__()
-> idxs = try_int(idxs)
(Pdb) n
> /home/santhosr/.local/lib/python3.6/site-packages/fastai/data_block.py(94)__getitem__()
-> if isinstance(idxs, numbers.Integral): return self.get(idxs)
(

(Pdb) ll
612  	    def __next__(self):
613  	        if self.num_workers == 0:  # same-process loading
614  	            indices = next(self.sample_iter)  # may raise StopIteration
615  	            batch = self.collate_fn([self.dataset[i] for i in indices])
616  	            if self.pin_memory:
617  	                batch = pin_memory_batch(batch)
618  ->	            return batch
619  	
620  	        # check if the next sample has already been generated
621  	        if self.rcvd_idx in self.reorder_dict:
622  	            batch = self.reorder_dict.pop(self.rcvd_idx)
623  	            return self._process_next_batch(batch)
624  	
625  	        if self.batches_outstanding == 0:
626  	            self._shutdown_workers()
627  	            raise StopIteration
628  	
629  	        while True:
630  	            assert (not self.shutdown and self.batches_outstanding > 0)
631  	            idx, batch = self._get_batch()
632  	            self.batches_outstanding -= 1
633  	            if idx

In [13]:
pdb.run('learn.predict(data.valid_dl.x[1])')

> <string>(1)<module>()->None
(Pdb) s
--Call--
> /home/santhosr/.local/lib/python3.6/site-packages/fastai/basic_data.py(35)__getattr__()
-> def __getattr__(self,k:str)->Any: return getattr(self.dl, k)
(Pdb) n
> /home/santhosr/.local/lib/python3.6/site-packages/fastai/basic_data.py(35)__getattr__()
-> def __getattr__(self,k:str)->Any: return getattr(self.dl, k)
(Pdb) n
--Return--
> /home/santhosr/.local/lib/python3.6/site-packages/fastai/basic_data.py(35)__getattr__()->ImageItemList.../ProcessedData
-> def __getattr__(self,k:str)->Any: return getattr(self.dl, k)
(Pdb) n
--Call--
> /home/santhosr/.local/lib/python3.6/site-packages/fastai/data_block.py(92)__getitem__()
-> def __getitem__(self,idxs:int)->Any:
(Pdb) n
> /home/santhosr/.local/lib/python3.6/site-packages/fastai/data_block.py(93)__getitem__()
-> idxs = try_int(idxs)
(Pdb) n
> /home/santhosr/.local/lib/python3.6/site-packages/fastai/data_block.py(94)__getitem__()
-> if isinstance(idxs, numbers.Integral): return self.get(idxs)
(

(Pdb) s
> /home/santhosr/.conda/envs/fastai/lib/python3.6/site-packages/torch/utils/data/dataloader.py(523)__init__()
-> self.pin_memory = loader.pin_memory and torch.cuda.is_available()
(Pdb) n
> /home/santhosr/.conda/envs/fastai/lib/python3.6/site-packages/torch/utils/data/dataloader.py(524)__init__()
-> self.timeout = loader.timeout
(Pdb) n
> /home/santhosr/.conda/envs/fastai/lib/python3.6/site-packages/torch/utils/data/dataloader.py(526)__init__()
-> self.sample_iter = iter(self.batch_sampler)
(Pdb) n
> /home/santhosr/.conda/envs/fastai/lib/python3.6/site-packages/torch/utils/data/dataloader.py(528)__init__()
-> base_seed = torch.LongTensor(1).random_().item()
(Pdb) n
> /home/santhosr/.conda/envs/fastai/lib/python3.6/site-packages/torch/utils/data/dataloader.py(530)__init__()
-> if self.num_workers > 0:
(Pdb) n
--Return--
> /home/santhosr/.conda/envs/fastai/lib/python3.6/site-packages/torch/utils/data/dataloader.py(530)__init__()->None
-> if self.num_workers > 0:
(Pdb) n
--Return--

Exception ignored in: <generator object DeviceDataLoader.__iter__ at 0x7f473d59f0a0>
Traceback (most recent call last):
  File "/home/santhosr/.local/lib/python3.6/site-packages/fastai/basic_data.py", line 70, in __iter__
    for b in self.dl: yield self.proc_batch(b)
  File "/home/santhosr/.conda/envs/fastai/lib/python3.6/bdb.py", line 57, in trace_dispatch
    return self.dispatch_exception(frame, arg)
  File "/home/santhosr/.conda/envs/fastai/lib/python3.6/bdb.py", line 122, in dispatch_exception
    if self.quitting: raise BdbQuit
bdb.BdbQuit: 


In [ ]:
pdb.run('learn.predict(data.valid_dl.x[1])')

> <string>(1)<module>()->None
(Pdb) s
--Call--
> /home/santhosr/.local/lib/python3.6/site-packages/fastai/basic_data.py(35)__getattr__()
-> def __getattr__(self,k:str)->Any: return getattr(self.dl, k)
(Pdb) n
> /home/santhosr/.local/lib/python3.6/site-packages/fastai/basic_data.py(35)__getattr__()
-> def __getattr__(self,k:str)->Any: return getattr(self.dl, k)
(Pdb) n
--Return--
(Pdb) 
> /home/santhosr/.local/lib/python3.6/site-packages/fastai/basic_data.py(35)__getattr__()->ImageItemList.../ProcessedData
-> def __getattr__(self,k:str)->Any: return getattr(self.dl, k)
--Call--
> /home/santhosr/.local/lib/python3.6/site-packages/fastai/data_block.py(92)__getitem__()
-> def __getitem__(self,idxs:int)->Any:
(Pdb) n
> /home/santhosr/.local/lib/python3.6/site-packages/fastai/data_block.py(93)__getitem__()
-> idxs = try_int(idxs)
(Pdb) n
> /home/santhosr/.local/lib/python3.6/site-packages/fastai/data_block.py(94)__getitem__()
-> if isinstance(idxs, numbers.Integral): return self.get(idxs)
(P

(Pdb) n
> /home/santhosr/.local/lib/python3.6/site-packages/fastai/basic_data.py(142)one_batch()
-> finally: self.num_workers = w
(Pdb) n
> /home/santhosr/.local/lib/python3.6/site-packages/fastai/basic_data.py(143)one_batch()
-> if detach: x,y = to_detach(x,cpu=cpu),to_detach(y,cpu=cpu)
(Pdb) p detach
False
(Pdb) s
> /home/santhosr/.local/lib/python3.6/site-packages/fastai/basic_data.py(144)one_batch()
-> norm = getattr(self,'norm',False)
(Pdb) p norm
*** NameError: name 'norm' is not defined
(Pdb) s
> /home/santhosr/.local/lib/python3.6/site-packages/fastai/basic_data.py(145)one_batch()
-> if denorm and norm:
(Pdb) p norm
functools.partial(<function _normalize_batch at 0x7f4751c2cd08>, mean=tensor([0.7601, 0.7601, 0.7601]), std=tensor([0.3307, 0.3307, 0.3307]), do_x=True, do_y=False)
(Pdb) ll
136  	    def one_batch(self, ds_type:DatasetType=DatasetType.Train, detach:bool=True, denorm:bool=True, cpu:bool=True)->Collection[Tensor]:
137  	        "Get one batch from the data loader of 

In [ ]:
n